In [ ]:
import brightway2 as bw
import pandas as pd
import numpy as np

In [ ]:
bw.projects.set_current('GSA for oases') #TODO choose project
co = bw.Database('CH consumption 1.0')
average_consump = co.search('ch hh average consumption')[0]
average_consump

In [ ]:
list_functional_units = []
for exc in average_consump.exchanges():
    if exc['type'] != 'production':
        list_functional_units.append({exc['input']: exc['amount']})
        
list_methods = [('IPCC 2013', 'climate change', 'GWP 100a')]

In [ ]:
%%time
bw.calculation_setups['av_consump_contribution_analysis'] = {'inv':list_functional_units, 'ia':list_methods}
myMultiLCA = bw.MultiLCA('av_consump_contribution_analysis')
lcia_unit = bw.Method(list_methods[0]).metadata['unit']
fus = [bw.get_activity(list(el.keys())[0])['name'][:25] for el in list_functional_units]
df = pd.DataFrame(index=fus, columns=[lcia_unit], data=myMultiLCA.results)
df['exchanges'] = [list(fu.values())[0] for fu in list_functional_units]
df['units'] = [bw.get_activity(list(el.keys())[0])['unit'] for el in list_functional_units]
df.sort_values(lcia_unit, ascending=False, inplace=True)
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(df)

# Biosphere

In [ ]:
import bw2analyzer

In [ ]:
lca = bw.LCA({average_consump: 1}, list_methods[0])

In [ ]:
lca.lci()
lca.lcia()
lca.score

In [ ]:
ca = bw2analyzer.ContributionAnalysis()

In [ ]:
ca.annotated_top_processes(lca)

# Contribution analysis for sectors

In [ ]:
import brightway2 as bw
import pandas as pd
import plotly.graph_objects as go

In [ ]:
%%time
project = 'GSA for oases'
bw.projects.set_current(project)
co = bw.Database("CH consumption 1.0")
demand_act = co.search('ch hh average consumption')
assert len(demand_act) == 1
demand = {demand_act[0]: 1}
list_methods = [('IPCC 2013', 'climate change', 'GWP 100a')]
sectors = [act for act in co if 'sector' in act["name"]]
list_fus = []
for act in sectors:
    list_fus.append({act: 1})
    
bw.calculation_setups['sector_contribution_analysis'] = {'inv':list_fus, 'ia':list_methods}
myMultiLCA = bw.MultiLCA('sector_contribution_analysis')
lcia_unit = bw.Method(list_methods[0]).metadata['unit']
fus = [bw.get_activity(list(el.keys())[0])['name'][:] for el in list_fus]
df = pd.DataFrame(index=fus, columns=[lcia_unit], data=myMultiLCA.results)
df['units'] = [bw.get_activity(list(el.keys())[0])['unit'] for el in list_fus]
df.sort_values(lcia_unit, ascending=False, inplace=True)

# with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
#     print(df)

In [ ]:
df_plot = df.iloc[:7]
y = df_plot.index.values
y = [el.replace('sector','') for el in y]
y[1] = 'Housing, water, electricity, gas'
y[3] = 'Durable goods (computers, printers)'
x = df_plot['kg CO2-Eq'].values

In [ ]:
fig = go.Figure()
fig.add_trace(
    go.Bar(x=x,y=y,orientation='h',)
)
fig.update_layout(
    height=200,
    width=450,
    margin=dict(t=0,b=0,l=250,r=0),
    yaxis=dict(autorange="reversed"),
    xaxis_title='LCIA scores, [kg CO2-eq]'
)


In [ ]:
fig.write_image('contribution_analysis_esd.pdf')

In [ ]:
list(sectors[8].exchanges())[1].as_dict()

In [ ]:
sectors